In [1]:
import torch
import math
import matplotlib.pyplot as plt
import gpytorch
%matplotlib inline
%load_ext autoreload
%autoreload 2
import time

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import pandas as pd

C:\Users\jmadinn\AppData\Local\Temp\ipykernel_30032\2537344704.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
torch.cuda.empty_cache()
start = time.time()
if (torch.cuda.is_available()):
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [4]:
fineSpringConstants = pd.read_csv("fineSpringParameters.csv",header=None)

In [5]:
timesteps = pd.read_csv("timesteps.csv",header=None)

In [6]:
coarseSpringConstants = np.arange(0.1,51.1,5)
mediumSpringConstants = np.arange(0.1,51.1,1)

In [7]:
fineDisplacements = pd.read_csv("fineDisplacements.csv",header=None)

In [8]:
fineDisplacements.set_index(fineSpringConstants.to_numpy().squeeze(),inplace=True)

In [9]:
fineDisplacements.columns = timesteps.to_numpy().squeeze()

In [10]:
coarseDisplacements = fineDisplacements.loc[coarseSpringConstants]

In [11]:
mediumDisplacements = fineDisplacements.loc[mediumSpringConstants]

In [12]:
inputFeatures = np.meshgrid(timesteps, mediumSpringConstants)

In [13]:
xt = np.stack((inputFeatures[1],inputFeatures[0]),axis=-1).reshape((-1,2))

In [14]:
train_x = torch.from_numpy(xt)
#train_x = train_x.to(device)

In [15]:
yt = mediumDisplacements.to_numpy().flatten()

In [16]:
train_y = torch.from_numpy(yt)
#train_y = train_y.to(device)

In [17]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
#likelyhood = likelihood.to(device)
model = ExactGPModel(train_x, train_y, likelihood)
#model = model.to(device)

In [18]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 0.934   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.892   lengthscale: 0.744   noise: 0.644
Iter 3/50 - Loss: 0.848   lengthscale: 0.798   noise: 0.598
Iter 4/50 - Loss: 0.804   lengthscale: 0.854   noise: 0.554
Iter 5/50 - Loss: 0.761   lengthscale: 0.911   noise: 0.513
Iter 6/50 - Loss: 0.718   lengthscale: 0.970   noise: 0.474
Iter 7/50 - Loss: 0.675   lengthscale: 1.031   noise: 0.437
Iter 8/50 - Loss: 0.634   lengthscale: 1.092   noise: 0.402
Iter 9/50 - Loss: 0.592   lengthscale: 1.152   noise: 0.370
Iter 10/50 - Loss: 0.545   lengthscale: 1.212   noise: 0.340
Iter 11/50 - Loss: 0.503   lengthscale: 1.269   noise: 0.312
Iter 12/50 - Loss: 0.461   lengthscale: 1.322   noise: 0.285
Iter 13/50 - Loss: 0.416   lengthscale: 1.371   noise: 0.261
Iter 14/50 - Loss: 0.374   lengthscale: 1.415   noise: 0.238
Iter 15/50 - Loss: 0.331   lengthscale: 1.454   noise: 0.217
Iter 16/50 - Loss: 0.285   lengthscale: 1.488   noise: 0.198
Iter 17/50 - Loss: 0.238   length

In [19]:
testFeatures = np.meshgrid(timesteps,fineSpringConstants)

In [20]:
test_x = torch.from_numpy(np.stack((testFeatures[1],testFeatures[0]),axis=-1).reshape((-1,2)))
test_x.to(device)

tensor([[ 0.1000,  0.0000],
        [ 0.1000,  0.1000],
        [ 0.1000,  0.2000],
        ...,
        [50.1000, 59.8000],
        [50.1000, 59.9000],
        [50.1000, 60.0000]], device='cuda:0', dtype=torch.float64)

In [21]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(): #gpytorch.settings.fast_pred_var():
    #test_x = torch.linspace(0, 1, 51)
    observed_pred = likelihood(model(test_x))

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 73832374008 bytes.

In [ ]:
end = time.time()
print("Time elapsed",end-start)

In [ ]:
with torch.no_grad():
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    #fig = plt.figure()
    #ax = Axes3D(fig)
    surf = ax.plot_trisurf(test_x[0:,0].numpy(), test_x[0:,1].numpy(),observed_pred.mean.numpy(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    #plt.imshow(z)
    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)
    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.scatter(train_x[0:,0].numpy(),train_x[0:,1].numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    #ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    #ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    #ax.set_ylim([-3, 3])
    ax.legend([ 'Mean','Observed Data'])
    plt.show()
    

In [22]:
73832374008/1024

72101927.7421875

In [23]:
73832374008/(1024*1024)

70412.03881072998